# Imports and loading dataset 

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

In [2]:
df = pd.read_excel("ANZ synthesised transaction dataset.xlsx")

In [4]:
df.head()

,status,card_present_flag,bpay_biller_code,account,currency,long_lat,txn_description,merchant_id,merchant_code,first_name,...,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,country,customer_id,merchant_long_lat,movement
0,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,POS,81c48296-73be-44a7-befa-d053f48ce7cd,NaN,Diana,...,26,Ashmore,QLD,2018-08-01T01:01:15.000+0000,16.25,a623070bfead4541a6b0fff8a09e706c,Australia,CUS-2487424745,153.38 -27.99,debit
1,authorized,0.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,830a451c-316e-4a6a-bf25-e37caedca49e,NaN,Diana,...,26,Sydney,NSW,2018-08-01T01:13:45.000+0000,14.19,13270a2a902145da9db4c951e04b51b9,Australia,CUS-2487424745,151.21 -33.87,debit
2,authorized,1.0,NaN,ACC-1222300524,AUD,151.23 -33.94,POS,835c231d-8cdf-4e96-859d-e9d571760cf0,NaN,Michael,...,38,Sydney,NSW,2018-08-01T01:26:15.000+0000,6.42,feb79e7ecd7048a5a36ec889d1a94270,Australia,CUS-2142601169,151.21 -33.87,debit
3,authorized,1.0,NaN,ACC-1037050564,AUD,153.10 -27.66,SALES-POS,48514682-c78a-4a88-b0da-2d6302e64673,NaN,Rhonda,...,40,Buderim,QLD,2018-08-01T01:38:45.000+0000,40.90,2698170da3704fd981b15e64a006079e,Australia,CUS-1614226872,153.05 -26.68,debit
4,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,b4e02c10-0852-4273-b8fd-7b3395e32eb0,NaN,Diana,...,26,Mermaid Beach,QLD,2018-08-01T01:51:15.000+0000,3.25,329adf79878c4cf0aeb4188b4691c266,Australia,CUS-2487424745,153.44 -28.06,debit


### Modifying data to obtain salaries for each customer

In [5]:
df_salaries = df[df["txn_description"]=="PAY/SALARY"].groupby("customer_id").mean()
df_salaries.head()

,card_present_flag,merchant_code,balance,age,amount
customer_id,,,,,
CUS-1005756958,NaN,0.0,4718.665385,53,970.47
CUS-1117979751,NaN,0.0,11957.202857,21,3578.65
CUS-1140341822,NaN,0.0,5841.720000,28,1916.51
CUS-1147642491,NaN,0.0,8813.467692,34,1711.39
CUS-1196156254,NaN,0.0,23845.717143,34,3903.73


In [6]:
salaries = []

for customer_id in df["customer_id"]:
    salaries.append(int(df_salaries.loc[customer_id]["amount"]))
    
df["annual_salary"] = salaries

In [7]:
df_cus = df.groupby("customer_id").mean()
df_cus.head()

,card_present_flag,merchant_code,balance,age,amount,annual_salary
customer_id,,,,,,
CUS-1005756958,0.812500,0.0,2275.852055,53,222.862603,970
CUS-1117979751,0.826923,0.0,9829.929000,21,339.843700,3578
CUS-1140341822,0.815385,0.0,5699.212250,28,212.632500,1916
CUS-1147642491,0.750000,0.0,9032.841186,34,245.600169,1711
CUS-1196156254,0.785276,0.0,22272.433755,34,147.145796,3903


In [43]:
df_cus['balance'].corr(df_cus['annual_salary'])

0.20564308404356624

In [44]:
df_cus['balance'].corr(df_cus['age'])

0.231963096050887

In [45]:
df_cus['age'].corr(df_cus['annual_salary'])

-0.13522694954964612

# Predictive Analytics

### Linear Regression

In [24]:
N_train = int(len(df_cus)*0.7)
X_train = df_cus.drop("annual_salary", axis=1).iloc[:N_train]
Y_train = df_cus["annual_salary"].iloc[:N_train]
X_test = df_cus.drop("annual_salary", axis=1).iloc[N_train:]
Y_test = df_cus["annual_salary"].iloc[N_train:]

In [25]:
linear_reg = LinearRegression()

In [26]:
linear_reg.fit(X_train, Y_train)
linear_reg.score(X_train, Y_train)

0.2468897331961574

In [27]:
linear_reg.predict(X_test)

array([3132.72421023, 2175.39641246, 2098.83496666, 2648.18350248,
       1783.60238994, 2779.34238454, 2709.70361603, 2961.5936652 ,
       2244.52686912, 1665.19124951, 2038.4996552 , 2972.85574737,
       2006.76417873, 1846.04764406, 2291.3211743 , 2156.44498139,
       1879.88463891, 5508.09056187, 1970.1444943 , 2263.4707209 ,
       1764.79270295, 2744.0674848 , 2172.82833702, 3958.85094997,
       2337.95629146, 1839.16986473, 2173.64875609, 1559.47188362,
       1851.43350431, 2570.51455513])

In [28]:
linear_reg.score(X_test, Y_test)

-0.23854072913735003

As there is very little correlation between the columns the linear model would not be sufficient

### Decision Tree - Classification and Regression

In [29]:
df_cat = df[["txn_description", "gender", "age", "merchant_state", "movement"]]

In [30]:
pd.get_dummies(df_cat).head()

,age,txn_description_INTER BANK,txn_description_PAY/SALARY,txn_description_PAYMENT,txn_description_PHONE BANK,txn_description_POS,txn_description_SALES-POS,gender_F,gender_M,merchant_state_ACT,merchant_state_NSW,merchant_state_NT,merchant_state_QLD,merchant_state_SA,merchant_state_TAS,merchant_state_VIC,merchant_state_WA,movement_credit,movement_debit
0,26,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1
1,26,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1
2,38,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1
3,40,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1
4,26,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1


In [31]:
N_train = int(len(df)*0.7)
X_train = pd.get_dummies(df_cat).iloc[:N_train]
Y_train = df["annual_salary"].iloc[:N_train]
X_test = pd.get_dummies(df_cat).iloc[N_train:]
Y_test = df["annual_salary"].iloc[N_train:]

#### Classification

In [32]:
decision_tree_class = DecisionTreeClassifier()

In [33]:
decision_tree_class.fit(X_train, Y_train)
decision_tree_class.score(X_train, Y_train)

0.7909845788849348

In [34]:
decision_tree_class.predict(X_test)

array([2840, 1013,  996, ..., 4054, 1043,  996], dtype=int64)

In [35]:
decision_tree_class.score(X_test, Y_test)

0.7553279822861888

#### Regression

In [36]:
decision_tree_reg = DecisionTreeRegressor()

In [37]:
decision_tree_reg.fit(X_train, Y_train)
decision_tree_reg.score(X_train, Y_train)

0.7524228033767403

In [38]:
decision_tree_reg.predict(X_test)

array([3649.2       , 1012.4       , 1642.57894737, ..., 3446.625     ,
       1043.        , 1642.57894737])

In [40]:
decision_tree_reg.score(X_test, Y_test)

0.6816005207536532